# Imports

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   #if like me you do not have a lot of memory in your GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "" #then these two lines force keras to use your CPU
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization, LeakyReLU, Conv2DTranspose, ReLU, Reshape
# from keras.models import Sequential
# from keras.layers import Dense, 
from tensorflow.keras.utils import to_categorical

import random
import itertools
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sparse
  # not scipy sparse because that is not how michael encoded it

### Defocus parameters
Carina: Here are the defocus parameters, ordered the same as X and y. You could pull it into the Data Generator easily enough, but I'm not sure in Keras how to pass it in in the middle of the network.

In [ ]:
# Defocus params
with open(r"../em_data/defocus_list.pkl", 'rb') as f:
  defocus_list = pickle.load(f)

# load data - small size

In [ ]:
with open(r"../em_data/X_list_84x54x49.pkl", 'rb') as f:
  # despite the name, it has dimensions 84x54x98
  X_3d = pickle.load(f)

In [ ]:
with open(r"../em_data/y_list_84x54.pkl", 'rb') as f:
  y_2d = pickle.load(f)

In [ ]:
print(len(X_3d))
print(type(X_3d))
print(X_3d[0].shape)

In [ ]:
print(len(y_2d))
print(type(y_2d))
print(y_2d[0].shape)

In [ ]:
# every element in the X_3d list is a sparse matrix, convert it to a 3D numpy array
# takes too large of a memory, only process the first 3 matrix
X_array = []
for i in range(3):
  X_array.append(sparse.COO.todense(X_3d[i]))

In [ ]:
# change X to be an array of input shape (#_samples, x, y, z)
# change y to be an array of output shape (#_samples, x, y)
# also crop them to be squares/cubes
# but for the 3D, do not take the canter on the z-dim, take the front because those are closer to the camera

X_train = np.zeros(shape=(3, 32, 32, 32,1))
y_train = np.zeros(shape=(3,32,32,1))
  # must add a channel dimension even if we only have 1 channel
  # the keras model require the extra channel dimension to do 2d and 3d convolutions

for i in range(3):
  X_train[i, :, :, :, 0] = X_array[i][26:58, 11:43, 0:32]

for i in range(3):
  y_train[i, :, :, 0]  = y_2d[i][11:43, 26:58]

In [ ]:
np.unique(X_train)

In [ ]:
print(X_train.shape)
print(y_train.shape)

### DataGenerator

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, X_filename, y_filename, use_range, voxel_dim, img_dim, batch_size, shuffle=True):
        self.X_filename = X_filename
        self.y_filename = y_filename
        self.use_range = use_range
        self.voxel_dim = voxel_dim
        self.img_dim = img_dim
        self.shuffle = shuffle # Not implemented yet
        self.batch_size = batch_size
        self.X = self.load_pickle(X_filename) # List of arrays
        self.y = self.load_pickle(y_filename) # List of arrays

    def load_pickle(self,filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        first_idx = int(len(data)*self.use_range[0])
        last_idx =  int(len(data)*self.use_range[1])
        return data[first_idx:last_idx]

    def __len__(self):
        'Denotes the number of batches per epoch'
        sample_count = len(self.X)
        return int(sample_count / self.batch_size)

    def __getitem__(self, index):
        'Generate one batch of data'
        X = np.asarray([sparse.COO.todense(xi) for xi in self.X[index*self.batch_size:(index+1)*self.batch_size]])
        y = np.asarray(self.y[index*self.batch_size:(index+1)*self.batch_size])
        X = np.expand_dims(X[:,26:58, 11:43, 0:32],axis=-1) # Generalize this
        y = np.expand_dims(y[:,11:43, 26:58],axis=-1) # Generalize this
        # Adjust y to [0,1]
        y_train = y[:int(y.shape[0]*0.8)]
        y_adj = (y-y_train.min()) / (y_train.max()-y_train.min())
        return X, y_adj

In [ ]:
X_filename = '../em_data/X_list_84x54x49.pkl'
y_filename = '../em_data/y_list_84x54.pkl'
train_range = [0.,0.8]
valid_range = [0.8,0.9]
test_range = [0.9,1.]
train_generator = DataGenerator(X_filename,y_filename,train_range,32,32,5)
valid_generator = DataGenerator(X_filename,y_filename,valid_range,32,32,5)
test_generator = DataGenerator(X_filename,y_filename,test_range,32,32,5)

In [ ]:
X,y = train_generator[22]
print(X.shape)
print(y.shape)
print(y.min(),y.max())

# model setup

In [ ]:
 sample_shape = (32, 32, 32,1)

In [ ]:
# Create the model
model = Sequential()
model.add(Conv3D(filters=32, 
                 kernel_size=(4, 4, 4), 
                 strides=(2, 2, 2),
                 padding='same', 
                    # `same` just means as long as even just the left most 1 column of your kernel is still in the sample matrix, you will use padding to fill the parts that ran over the matrix and finish that mapping
                    # if you keep moving till you kernel does not overlap with your matrix at all we will stop and won't pad
                 use_bias=False,
                 input_shape=sample_shape))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(LeakyReLU(alpha=0.2))

model.add(Conv3D(filters=64, 
                 kernel_size=(4, 4, 4), 
                 strides=(2, 2, 2),
                 padding='same',
                 use_bias=False))
model.add(BatchNormalization(center=True, scale=True))
model.add(LeakyReLU(alpha=0.2))

model.add(Conv3D(filters=128, 
                 kernel_size=(4, 4, 4), 
                 strides=(2, 2, 2),
                 padding='same',
                 use_bias=False))
model.add(BatchNormalization(center=True, scale=True))
model.add(LeakyReLU(alpha=0.2))

model.add(Conv3D(filters=256, 
                 kernel_size=(4, 4, 4), 
                 strides=(2, 2, 2),
                 padding='same',
                 use_bias=False))
model.add(BatchNormalization(center=True, scale=True))
model.add(LeakyReLU(alpha=0.2))

model.add(Conv3D(filters=100, 
                 kernel_size=(2, 2, 2), 
                 strides=(1, 1, 1),
                 padding='valid',
                 use_bias=False))
model.add(LeakyReLU(alpha=0.2))


model.add(Reshape((1,1,100), input_shape=(1,1,1,100)))
  # must reshape from a 3-D structure with 100 channels to a 2-D image having 100 channels
  # so Conv2DTranspose can work properly


model.add(Conv2DTranspose(filters=256,
                          kernel_size=(2,2),
                          strides=(1,1),
                          padding='valid',
                          use_bias=False
                          ))
model.add(BatchNormalization(center=True, scale=True))
model.add(ReLU())

model.add(Conv2DTranspose(filters=128,
                          kernel_size=(4,4),
                          strides=(2,2),
                          padding='same',
                          use_bias=False
                          ))
model.add(BatchNormalization(center=True, scale=True))
model.add(ReLU())

model.add(Conv2DTranspose(filters=64,
                          kernel_size=(4,4),
                          strides=(2,2),
                          padding='same',
                          use_bias=False
                          ))
model.add(BatchNormalization(center=True, scale=True))
model.add(ReLU())

model.add(Conv2DTranspose(filters=32,
                          kernel_size=(4,4),
                          strides=(2,2),
                          padding='same',
                          use_bias=False
                          ))
model.add(BatchNormalization(center=True, scale=True))
model.add(ReLU())

model.add(Conv2DTranspose(filters=1,
                          kernel_size=(4,4),
                          strides=(2,2),
                          padding='same',
                          use_bias=False
                          ))
model.add(Dense(1, activation='tanh'))

# Variable LR for optimizer
# MHS Note: I tried a decaying learning rate here, but you can take that out if you prefer
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=100,
    decay_rate=0.85)

model.compile(loss='mean_squared_error',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule))
print("input shape:", sample_shape)
model.summary()

In [ ]:
# Fit with DataGenerators
model.fit(train_generator,
          validation_data = valid_generator,
          epochs = 200)

# # Fit data to model
# history = model.fit(X_train, y_train,
#                     batch_size=1,
#                     epochs=3,
#                     verbose=1,
#                     validation_split=0.3)

### Plot the images

In [ ]:
X_test,y_test = valid_generator[4] # Pick a random value (4 here) in valid_generator or train_generator
y_pred = model.predict(X_test)

In [ ]:
# Plots 5 samples from the chosen sample
plt.figure(figsize=[6,15])
for i in range(5):
    plt.subplot(5,2,2*i+1)
    if i==0:
        plt.title("True")
    plt.imshow(y_test[i,:,:,0],cmap='gray')
    plt.subplot(5,2,2*i+2)
    plt.imshow(y_pred[i,:,:,0],cmap='gray')
    if i==0:
        plt.title("Predicted")

### Plot the voxels

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.gca(projection='3d')
# ax.set_aspect('equal')

ax.voxels(X_test[0,:,:,:,0], edgecolor="k")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()